# Basic walkthrough

Here is a basic walkthrough of how to run the whole pipeline from preprocessing to training the GAN. The walkthrough is only one example to see what one can to to tune and tweek on different datasets, hyperparemeters and changing between models see `Tuning.ipynb`. Every script should be run(```# RUN```) from the repostitory folder.


-------
### Datasets
The combined dataset with CNN/Daily mail and Exabel data is used. It has 255157 articles with summaries.


### Preprocess datasets
#### CNN dataset
- The tokenized version of the CNN/Daily Mail dataset can be downloaded from [Dataset Link](https://github.com/JafferWilson/Process-Data-of-CNN-DailyMail).
- Create a folder `data` in the parent folder of the repo, on the same level as the repo. There is a datafolder in the repo with dummyfiles in the leafnodes you can move.
- Place the tokenized data in the `cnn_clean` folder


#### Exabel Dataset:
- Get from Erlend Aune in json-format
- Place in `data` folder

**save the exabel data in the same format as CNN/DM**
```sh
# RUN
python3 preprocess/clean_exa.py 
```

```sh
# RUN
python3 preprocess/combined_preprocess.py
```

**create vocabulary and preprocess all the articles from both the CNN, daily mail and exabel and save as pickle**
```sh
# RUN
python3 preprocess/preprocess_pointer.py
```

-------
### Generator pretraining
```sh
# RUN
python3 training/seq2seq/run_experiment.py training/seq2seq/experiments/combined_test_1 0
```
PS: it prints after every epoch. But logs more frequently in output.log in ```training/seq2seq/experiments/combined_test_1```


**time to run: **
40h ++


----------
### Generate fake data to train discriminator:

Creates samples using the generator as fake ones.
```sh
# RUN
python3 evaluation/seq2seq/generate_fake_sampled_data.py 0
```

- The data is saved in ```../data/cnn_validation_sampled_data```.
- Move the data over to the sibling folder: ```cnn_fake_data```.
- it is possible to have several fake_data files from different generators. Then evaluate a discriminator on different datasets.

---
### Generate real data to train discriminator:
```sh
# RUN
python3 evaluation/seq2seq/generate_real_data.py
```

- the real data is then saved in ```../data/cnn_real_data```

---
### Discriminator pretraining
- first specify name of real_data_file and fake_data_directory in ```training/classifier/experiments/cnn_test_1/config.json```

```sh
# RUN
python3 training/classifier/run_experiment.py training/classifier/experiments/cnn_test_1 0
```

- pretrains the Discriminator on the fake data and real data generated
- Saves the model in the same folder as the config.json file

---
### GAN training
```sh
# RUN
python3 training/GAN/run_experiment.py training/GAN/experiments/hb_cnn_test_1 0
```


---
### Evaluating

To evaluate generator models you yourself have to manually copy the models you want evaluated into a fitting ```models_first```-folder

#### Evaluate generator
To evaluate all the models through all epochs put them in a folder and call this:
```sh
# RUN
python3 evaluation/seq2seq/evaluate_multiple_generators.py output_for_eval/seq2seq/cnn/models_first 0
# or
python3 evaluation/seq2seq/evaluate_multiple_generators.py output_for_eval/gan/cnn/models_first 0
```
**time to run:** overnight

To calculate rouge for results of all the models:
```sh
# RUN
python3 evaluation/seq2seq/calculate_rouge_in_folder.py output_for_eval/seq2seq/cnn/models_first_eval
# or
python3 evaluation/seq2seq/calculate_rouge_in_folder.py output_for_eval/gan/cnn/models_first_eval
```

#### Evaluate Discriminator
```sh
# RUN
python3 evaluation/classifier/test_pretrained_classifier.py 0
```

Evaluates generater

**time to run:** seconds

---------
### Tensorboard
To see the statistics that are saved in ```log/``` during training.

```sh
 tensorboard --logdir log
```
